In [4]:
from pydantic import BaseModel, ConfigDict, Field, ValidationError

In [5]:
class Point2D(BaseModel):
    x: float = 0
    y: float = 0

In [6]:
class Circle2D(BaseModel):
    center: Point2D
    radius: float = Field(default=1, gt=0)

In [7]:
Circle2D(center=Point2D(x=1, y=1), radius=1)

Circle2D(center=Point2D(x=1.0, y=1.0), radius=1.0)

In [8]:
Circle2D(center=Point2D(x=1, y=1), radius=1).model_dump()

{'center': {'x': 1.0, 'y': 1.0}, 'radius': 1.0}

In [10]:
print(Circle2D(center=Point2D(x=1, y=1), radius=1).model_dump_json(indent=2))

{
  "center": {
    "x": 1.0,
    "y": 1.0
  },
  "radius": 1.0
}


In [11]:
data = {
    "center": {
        "x": 1.0,
        "y": 1.0
    },
    "radius": 1.0
}

In [12]:
Circle2D.model_validate(data)

Circle2D(center=Point2D(x=1.0, y=1.0), radius=1.0)

In [15]:
c = Circle2D.model_validate(data)
c

Circle2D(center=Point2D(x=1.0, y=1.0), radius=1.0)

In [16]:
c.center.x

1.0

In [17]:
json_data = """
{
    "firstName": "David",
    "lastName": "Hilbert",
    "contactInfo": {
        "email": "d.hilbert@spectral-theory.com",
        "homePhone": {
            "countryCode": 49,
            "areaCode": 551,
            "localPhoneNumber": 123456789
        }
    },
    "personalInfo": {
        "nationality": "German",
        "born": {
            "date": "1862-01-23",
            "place": {
                "city": "Konigsberg",
                "country": "Prussia"
            }
        },
        "died": {
            "date": "1943-02-14",
            "place": {
                "city": "Gottingen",
                "country": "Germany"
            }
        }
    },
    "awards": ["Lobachevsky Prize", "Bolyai Prize", "ForMemRS"],
    "notableStudents": ["von Neumann", "Weyl", "Courant", "Zermelo"]
}
"""

In [18]:
from pydantic import EmailStr, PastDate
from pydantic.alias_generators import to_camel

In [19]:
class ContactInfo(BaseModel):
    model_config = ConfigDict(extra="ignore")

    email: EmailStr | None = None

In [20]:
class PlaceInfo(BaseModel):
    city: str
    country: str

In [21]:
class PlaceDateInfo(BaseModel):
    model_config = ConfigDict(populate_by_name=True)
    date_: PastDate = Field(alias="date")
    place: PlaceInfo

In [22]:
class PersonalInfo(BaseModel):
    model_config = ConfigDict(extra="ignore")

    nationality: str
    born: PlaceDateInfo

In [24]:
class Person(BaseModel):
    model_config = ConfigDict(alias_generator=to_camel, populate_by_name=True, extra="ignore")

    first_name: str
    last_name: str
    contact_info: ContactInfo
    personal_info: PersonalInfo
    notable_students: list[str] = list


In [25]:
p = Person.model_validate_json(json_data)

In [26]:
p

Person(first_name='David', last_name='Hilbert', contact_info=ContactInfo(email='d.hilbert@spectral-theory.com'), personal_info=PersonalInfo(nationality='German', born=PlaceDateInfo(date_=datetime.date(1862, 1, 23), place=PlaceInfo(city='Konigsberg', country='Prussia'))), notable_students=['von Neumann', 'Weyl', 'Courant', 'Zermelo'])

In [28]:
from pprint import pprint
pprint(p.model_dump())

{'contact_info': {'email': 'd.hilbert@spectral-theory.com'},
 'first_name': 'David',
 'last_name': 'Hilbert',
 'notable_students': ['von Neumann', 'Weyl', 'Courant', 'Zermelo'],
 'personal_info': {'born': {'date_': datetime.date(1862, 1, 23),
                            'place': {'city': 'Konigsberg',
                                      'country': 'Prussia'}},
                   'nationality': 'German'}}


In [30]:
print(p.model_dump_json(indent=4))

{
    "first_name": "David",
    "last_name": "Hilbert",
    "contact_info": {
        "email": "d.hilbert@spectral-theory.com"
    },
    "personal_info": {
        "nationality": "German",
        "born": {
            "date_": "1862-01-23",
            "place": {
                "city": "Konigsberg",
                "country": "Prussia"
            }
        }
    },
    "notable_students": [
        "von Neumann",
        "Weyl",
        "Courant",
        "Zermelo"
    ]
}


In [31]:
from typing import Annotated
from pydantic import AfterValidator

In [32]:
SortedStringList = Annotated[
    list[str],
    AfterValidator(lambda v: sorted(v, key=str.casefold))
]

In [34]:
class Person(BaseModel):
    model_config = ConfigDict(alias_generator=to_camel, populate_by_name=True, extra="ignore")

    first_name: str
    last_name: str
    contact_info: ContactInfo
    personal_info: PersonalInfo
    notable_students: SortedStringList = Field(default_factory=list, repr=False)

In [35]:
p = Person.model_validate_json(json_data)


In [36]:
p

Person(first_name='David', last_name='Hilbert', contact_info=ContactInfo(email='d.hilbert@spectral-theory.com'), personal_info=PersonalInfo(nationality='German', born=PlaceDateInfo(date_=datetime.date(1862, 1, 23), place=PlaceInfo(city='Konigsberg', country='Prussia'))))

In [37]:
print(p.model_dump_json(indent=4))


{
    "first_name": "David",
    "last_name": "Hilbert",
    "contact_info": {
        "email": "d.hilbert@spectral-theory.com"
    },
    "personal_info": {
        "nationality": "German",
        "born": {
            "date_": "1862-01-23",
            "place": {
                "city": "Konigsberg",
                "country": "Prussia"
            }
        }
    },
    "notable_students": [
        "Courant",
        "von Neumann",
        "Weyl",
        "Zermelo"
    ]
}
